In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

23/02/05 00:08:54 WARN Utils: Your hostname, MacBook-Pro-de-Harrison.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
23/02/05 00:08:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/05 00:08:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_tweet = spark.read.json("../datalake/silver/twitter_datascience/tweet")

In [5]:
from pyspark.sql import functions as f

In [6]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- impression_count: long (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [15]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        )

In [13]:
tweet_conversas.orderBy("created_date").show()

+------------+--------+------+-------+-------+---------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|
+------------+--------+------+-------+-------+---------+
|  2023-02-04|      39|    18|      0|      0|     1007|
+------------+--------+------+-------+-------+---------+

